In [86]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt

# Definimos entidades a buscar a partir de scraping de ENISA
entity_list = pd.read_csv('df_enisa.csv', sep=';')
entity_list = entity_list["Marca"].to_list()

# Buscamos la fecha de hace 3 meses
today = dt.date.today()
fecha = today - dt.timedelta(days=90)

# Creamos la lista para recopilar tweets
tweets_list = []
tweets_enisa_list = []

for entity in entity_list:
    search = entity + " since:" + str(fecha)
    search_enisa = entity + " #clienteEnisa since:" + str(fecha)

    # Usamos TwitterSearchScraper para scrapear datos y guardar tweets en lista - Marca
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search).get_items()):
        if i>500:
            break
        tweets_list.append([entity, tweet.date, tweet.id, tweet.content, tweet.user.username])

    # Usamos TwitterSearchScraper para scrapear datos y guardar tweets en lista - menciones enisa
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_enisa).get_items()):
        if i>500:
            break
        tweets_enisa_list.append([entity, tweet.date, tweet.id, tweet.content, tweet.user.username])

# Creamos dataframes con toda la inforamción - por si potencialmente se quieren trabajar los datos
df_tweets = pd.DataFrame(tweets_list, columns=['Marca', 'Datetime', 'Tweet Id', 'Text', 'Username'])
df_tweets_enisa = pd.DataFrame(tweets_enisa_list, columns=['Marca', 'Datetime', 'Tweet Id', 'Text', 'Username'])

# Creamos dataframes con indicador de tweets en últimos 90d
df_90d = df_tweets.Marca.value_counts().to_frame(name='90d_enisa')
df_90d['Marca'] = df_90d.index
df_90d.reset_index(inplace=True, drop=True)

df_90d_enisa = df_tweets_enisa.Marca.value_counts().to_frame(name='90d_enisa')
df_90d_enisa['Marca'] = df_90d_enisa.index
df_90d_enisa.reset_index(inplace=True, drop=True)

# Generamos los ficheros de salida CSV
df_90d.to_csv('df_twitter_90d.csv', sep=";", na_rep= "",)
df_90d_enisa.to_csv('df_twitter_90d_enisa.csv', sep=";", na_rep= "",)
